### Working with Merged County Data

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import scipy.stats as st
import sklearn as sk

In [2]:
# Primary file -- merged_county_data
infile = 'Source_Data/merged2_county_data.csv'
cnty_df = pd.read_csv(infile)
cnty_df.head()

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
0,0,Autauga,Alabama,"Autauga County, AL",1001.0,163.0,55416.0,294.1,1001.0,19.0,...,9.0,42.0,11.7,14.0,"Autauga County, AL",1001.0,17103.72,1.158,9.282,0.276
1,1,Baldwin,Alabama,"Baldwin County, AL",1003.0,640.0,208563.0,306.9,1003.0,17.0,...,11.0,73.0,10.3,11.0,"Baldwin County, AL",1003.0,17660.59,1.158,9.282,0.276
2,2,Barbour,Alabama,"Barbour County, AL",1005.0,90.0,25965.0,346.6,1005.0,22.0,...,13.0,39.0,11.5,18.0,"Barbour County, AL",1005.0,17315.13,1.158,9.282,0.276
3,3,Bibb,Alabama,"Bibb County, AL",1007.0,79.0,22643.0,348.9,1007.0,20.0,...,10.0,57.0,11.2,15.0,"Bibb County, AL",1007.0,16904.87,1.158,9.282,0.276
4,4,Blount,Alabama,"Blount County, AL",1009.0,199.0,57704.0,344.9,1009.0,20.0,...,12.0,23.0,11.7,14.0,"Blount County, AL",1009.0,16393.63,1.158,9.282,0.276


In [3]:
# Let's check out how 'full' the data set is...
fill = pd.DataFrame({'Entries':cnty_df.count(),'Unique Items':cnty_df.nunique()})
fill

,Entries,Unique Items
Unnamed: 0,3252,3252
county,3189,1877
state,3244,56
county_and_state_x,2968,2968
code,2968,2968
Deaths,2968,725
Population,2968,2915
Crude Rate,2968,2186
fips,3197,3191
smoking,3197,33


* 3257 rows is more than any one item's count, so we didn't lose anything, but there were imperfect mergers.
* There are 3194 entries for county, and 56 for state.  Let's look at the state names:

In [4]:
cnty_df.state.value_counts(dropna=False)

Texas                   255
Virginia                171
Georgia                 160
Kentucky                121
Missouri                117
Kansas                  106
Illinois                103
North Carolina          101
Iowa                    100
Tennessee                96
Indiana                  96
Nebraska                 94
Ohio                     89
Minnesota                88
Michigan                 84
Mississippi              83
Oklahoma                 78
Arkansas                 76
Wisconsin                73
Pennsylvania             69
Alabama                  68
South Dakota             68
Florida                  68
Louisiana                66
Colorado                 65
New York                 63
California               59
Montana                  57
West Virginia            56
North Dakota             54
South Carolina           47
Idaho                    45
Washington               40
Oregon                   37
New Mexico               34
Alaska              

* Just 8 rows where we are missing a state.  Let's look at those:

In [5]:
nostate = cnty_df.loc[cnty_df.state.isnull(),:]
nostate

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,99,"District of Columbia, DC",NaN,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,NaN,NaN,NaN
101,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Dade County, FL",12025.0,19744.21,NaN,NaN,NaN
102,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Yellowstone National Park, MT",30113.0,16179.86,NaN,NaN,NaN
103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
105,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Clifton Forge city, VA",51560.0,15616.58,NaN,NaN,NaN
106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"South Boston city, VA",51780.0,16153.98,NaN,NaN,NaN


* These 8 are special cases that we can handle one by one now.  
* Next, let's look for NaN's in county names...

In [6]:
nocnty = cnty_df.loc[cnty_df.county.isnull(),:]
nocnty

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
67,67,NaN,Alabama,NaN,NaN,NaN,NaN,NaN,1000.0,22.0,...,11.0,65.0,11.0,14.0,NaN,NaN,NaN,1.158000,9.282000,0.276000
83,83,NaN,Alaska,NaN,NaN,NaN,NaN,NaN,2000.0,19.0,...,16.0,90.0,NaN,8.0,NaN,NaN,NaN,2.056000,23.180000,0.806000
100,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,NaN,NaN,NaN
101,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Dade County, FL",12025.0,19744.21,NaN,NaN,NaN
102,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Yellowstone National Park, MT",30113.0,16179.86,NaN,NaN,NaN
103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
105,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Clifton Forge city, VA",51560.0,15616.58,NaN,NaN,NaN
106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"South Boston city, VA",51780.0,16153.98,NaN,NaN,NaN
199,199,NaN,Indiana,NaN,NaN,NaN,NaN,NaN,18000.0,21.0,...,9.0,67.0,11.8,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000


* For these 63 cases, we have 51 excess "state" names where there are just stats for the whole state.  In those cases we have everything but sunlight.
* Of the remaining 12 cases, there are 5 -- regions + total US, where we just have aggregated drug use data.
* That leaves 7 extra cases, which are all part of the "special cases" we found earlier

In [7]:
# 1.  Miami-Dade County is the same as Dade County, we need to merge those rows, but we can't do it on codes because
# Miami-Dade has fips 12086 while the old Dade County has fips 12025.  But we can find them
cnty_df.loc[cnty_df.county_and_state_y == 'Dade County, FL',:]

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
101,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Dade County, FL",12025.0,19744.21,NaN,NaN,NaN


In [8]:
# The other row
cnty_df.loc[cnty_df.county == 'Miami-Dade',:]

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
867,867,Miami-Dade,Florida,"Miami-Dade County, FL",12086.0,7255.0,2712945.0,267.4,12086.0,14.0,...,19.0,80.0,7.2,8.0,NaN,NaN,NaN,1.869583,13.364583,0.269583


In [9]:
# If we just move three values (columns 17 to 19) to row 867 from row 101, we are good.
for n in range(17,20):
    cnty_df.iat[867,n] = cnty_df.iat[101,n]
# Then delete row 101
cnty_df = cnty_df.drop(index=101,axis=0)
cnty_df.iloc[99:110]

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,99,"District of Columbia, DC",NaN,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,NaN,NaN,NaN
102,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Yellowstone National Park, MT",30113.0,16179.86,NaN,NaN,NaN
103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
105,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Clifton Forge city, VA",51560.0,15616.58,NaN,NaN,NaN
106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"South Boston city, VA",51780.0,16153.98,NaN,NaN,NaN
107,107,De Kalb,Indiana,"De Kalb County, IN",18033.0,114.0,42746.0,266.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.44
108,108,Lagrange,Indiana,"Lagrange County, IN",18087.0,87.0,39110.0,222.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.44
109,109,La Porte,Indiana,"La Porte County, IN",18091.0,404.0,110015.0,367.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.44


In [10]:
cnty_df.iat[3245,2]

'District of Columbia'

In [11]:
#Next is the District of Columbia.  Same thing, just rows 100 and 99
for n in range(17,20):
    cnty_df.iat[99,n] = cnty_df.iat[100,n]
# Then copy the drug data from row 3245
for n in range(20,23):
    cnty_df.iat[99,n] = cnty_df.iat[3245,n]
# Then delete row 101
cnty_df = cnty_df.drop(index=100,axis=0)
cnty_df.iloc[99:105,:]

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,99,"District of Columbia, DC",NaN,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,3.425556,24.177778,0.472222
102,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Yellowstone National Park, MT",30113.0,16179.86,NaN,NaN,NaN
103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
105,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Clifton Forge city, VA",51560.0,15616.58,NaN,NaN,NaN
106,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"South Boston city, VA",51780.0,16153.98,NaN,NaN,NaN


In [12]:
# Next is Yellowstone National Park.  Turns out that county was merged into another county, so we can just delete it...
cnty_df = cnty_df.drop(index=102,axis=0)
# Same thing for South Boston
cnty_df = cnty_df.drop(index=105,axis=0)
# ... and Clifton Forge
cnty_df = cnty_df.drop(index=106,axis=0)
cnty_df.iloc[99:110,:]

,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,99,"District of Columbia, DC",NaN,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,3.425556,24.177778,0.472222
103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN
104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
107,107,De Kalb,Indiana,"De Kalb County, IN",18033.0,114.0,42746.0,266.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
108,108,Lagrange,Indiana,"Lagrange County, IN",18087.0,87.0,39110.0,222.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
109,109,La Porte,Indiana,"La Porte County, IN",18091.0,404.0,110015.0,367.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
110,110,Adams,Indiana,"Adams County, IN",18001.0,104.0,35232.0,295.2,18001.0,21.0,...,12.0,34.0,12.5,NaN,"Adams County, IN",18001.0,14630.84,1.167778,13.688889,0.440000
111,111,Allen,Indiana,"Allen County, IN",18003.0,937.0,370404.0,253,18003.0,19.0,...,9.0,65.0,12.6,NaN,"Allen County, IN",18003.0,14540.99,1.167778,13.688889,0.440000
112,112,Bartholomew,Indiana,"Bartholomew County, IN",18005.0,226.0,81402.0,277.6,18005.0,18.0,...,10.0,75.0,11.2,NaN,"Bartholomew County, IN",18005.0,15055.18,1.167778,13.688889,0.440000
113,113,Benton,Indiana,"Benton County, IN",18007.0,23.0,8650.0,265.9,18007.0,21.0,...,12.0,NaN,11.0,NaN,"Benton County, IN",18007.0,14809.22,1.167778,13.688889,0.440000


In [13]:
# Now just deal with Bedford, VA and Shannon, SD.
# For Shannon, SD, we'll just merge as we did before, but first we will reset the index to make it easier...
cnty_df = cnty_df.reset_index()
cnty_df.head()

,index,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
0,0,0,Autauga,Alabama,"Autauga County, AL",1001.0,163.0,55416.0,294.1,1001.0,...,9.0,42.0,11.7,14.0,"Autauga County, AL",1001.0,17103.72,1.158,9.282,0.276
1,1,1,Baldwin,Alabama,"Baldwin County, AL",1003.0,640.0,208563.0,306.9,1003.0,...,11.0,73.0,10.3,11.0,"Baldwin County, AL",1003.0,17660.59,1.158,9.282,0.276
2,2,2,Barbour,Alabama,"Barbour County, AL",1005.0,90.0,25965.0,346.6,1005.0,...,13.0,39.0,11.5,18.0,"Barbour County, AL",1005.0,17315.13,1.158,9.282,0.276
3,3,3,Bibb,Alabama,"Bibb County, AL",1007.0,79.0,22643.0,348.9,1007.0,...,10.0,57.0,11.2,15.0,"Bibb County, AL",1007.0,16904.87,1.158,9.282,0.276
4,4,4,Blount,Alabama,"Blount County, AL",1009.0,199.0,57704.0,344.9,1009.0,...,12.0,23.0,11.7,14.0,"Blount County, AL",1009.0,16393.63,1.158,9.282,0.276


In [14]:
# Check where the new rows are
cnty_df.loc[cnty_df.county_and_state_x == 'Shannon County, SD',:]

,index,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
352,357,357,Shannon,South Dakota,"Shannon County, SD",46113.0,17.0,14415.0,117.9 (Unreliable),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.978333,11.028333,0.248333


In [15]:
cnty_df.loc[cnty_df.county_and_state_y == 'Shannon County, SD',:]

,index,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
100,103,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Shannon County, SD",46113.0,15680.32,NaN,NaN,NaN


In [16]:
for n in range(17,20):
    cnty_df.iat[352,n] = cnty_df.iat[100,n]
# Then delete row 101
cnty_df = cnty_df.drop(index=100,axis=0)
cnty_df.iloc[99:105,:]

,index,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,99,99,"District of Columbia, DC",NaN,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,3.425556,24.177778,0.472222
101,104,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Bedford city, VA",51515.0,16091.55,NaN,NaN,NaN
102,107,107,De Kalb,Indiana,"De Kalb County, IN",18033.0,114.0,42746.0,266.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
103,108,108,Lagrange,Indiana,"Lagrange County, IN",18087.0,87.0,39110.0,222.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
104,109,109,La Porte,Indiana,"La Porte County, IN",18091.0,404.0,110015.0,367.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
105,110,110,Adams,Indiana,"Adams County, IN",18001.0,104.0,35232.0,295.2,18001.0,...,12.0,34.0,12.5,NaN,"Adams County, IN",18001.0,14630.84,1.167778,13.688889,0.440000


In [17]:
# Last but not least is Bedford city, VA ... note there could be a Bedford county in VA also ...
# Let's find all Bedford counties ...
cnty_df.loc[cnty_df.county == 'Bedford',:]

,index,Unnamed: 0,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
288,293,293,Bedford,Pennsylvania,"Bedford County, PA",42009.0,200.0,48325.0,413.9,42009.0,...,8.0,25.0,10.6,13.0,"Bedford County, PA",42009.0,14434.89,1.624286,12.271429,0.619286
462,467,467,Bedford,Virginia,"Bedford County, VA",51019.0,233.0,77960.0,298.9,51019.0,...,9.0,71.0,9.2,14.0,"Bedford County, VA",51019.0,16053.66,1.455000,11.498333,0.401667
2653,2658,2658,Bedford,Tennessee,"Bedford County, TN",47003.0,172.0,47484.0,362.2,47003.0,...,13.0,38.0,10.5,12.0,"Bedford County, TN",47003.0,15851.13,1.488750,11.655000,0.268750


In [18]:
# OK, only one Bedford County, VA.  After some research ... turns out Bedford city joined Bedford county in 2013
# so we can just delete it.
cnty_df = cnty_df.drop(index=101,axis=0)
cnty_df = cnty_df.reset_index()

In [19]:
# Now let's clear out some unused columns ...
cnty_df = cnty_df.drop(columns = ['level_0','index','Unnamed: 0'])
cnty_df.head()

,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,obesity,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
0,Autauga,Alabama,"Autauga County, AL",1001.0,163.0,55416.0,294.1,1001.0,19.0,38.0,...,9.0,42.0,11.7,14.0,"Autauga County, AL",1001.0,17103.72,1.158,9.282,0.276
1,Baldwin,Alabama,"Baldwin County, AL",1003.0,640.0,208563.0,306.9,1003.0,17.0,31.0,...,11.0,73.0,10.3,11.0,"Baldwin County, AL",1003.0,17660.59,1.158,9.282,0.276
2,Barbour,Alabama,"Barbour County, AL",1005.0,90.0,25965.0,346.6,1005.0,22.0,44.0,...,13.0,39.0,11.5,18.0,"Barbour County, AL",1005.0,17315.13,1.158,9.282,0.276
3,Bibb,Alabama,"Bibb County, AL",1007.0,79.0,22643.0,348.9,1007.0,20.0,38.0,...,10.0,57.0,11.2,15.0,"Bibb County, AL",1007.0,16904.87,1.158,9.282,0.276
4,Blount,Alabama,"Blount County, AL",1009.0,199.0,57704.0,344.9,1009.0,20.0,34.0,...,12.0,23.0,11.7,14.0,"Blount County, AL",1009.0,16393.63,1.158,9.282,0.276


In [20]:
# Last fix, put "DC" as the state for DC, and copy the drug data there too
cnty_df.iat[99,1] = 'District of Columbia'
cnty_df.iloc[99:102,:]

,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,obesity,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
99,"District of Columbia, DC",District of Columbia,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,3.425556,24.177778,0.472222
100,De Kalb,Indiana,"De Kalb County, IN",18033.0,114.0,42746.0,266.7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000
101,Lagrange,Indiana,"Lagrange County, IN",18087.0,87.0,39110.0,222.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.167778,13.688889,0.440000


In [21]:
# Last, let's get rid of the duplicate row 3239 for DC...
cnty_df = cnty_df.drop(index = 3239, axis = 0)
cnty_df.tail(10)

,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,obesity,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
3234,Uinta,Wyoming,"Uinta County, WY",56041.0,42.0,20773.0,202.2,56041.0,18.0,32.0,...,13.0,39.0,5.9,9.0,"Uinta County, WY",56041.0,17123.76,1.331,10.965,0.234
3235,Washakie,Wyoming,"Washakie County, WY",56043.0,18.0,8235.0,218.6 (Unreliable),56043.0,16.0,30.0,...,17.0,49.0,4.8,11.0,"Washakie County, WY",56043.0,16171.03,1.331,10.965,0.234
3236,Weston,Wyoming,"Weston County, WY",56045.0,21.0,7236.0,290.2,56045.0,17.0,31.0,...,14.0,41.0,4.1,9.0,"Weston County, WY",56045.0,15923.65,1.331,10.965,0.234
3237,NaN,Wyoming,NaN,NaN,NaN,NaN,NaN,56000.0,19.0,29.0,...,13.0,68.0,4.8,9.0,NaN,NaN,NaN,1.331,10.965,0.234
3238,Niobrara,Wyoming,NaN,NaN,NaN,NaN,NaN,56027.0,17.0,25.0,...,16.0,40.0,3.3,8.0,"Niobrara County, WY",56027.0,16054.19,1.331,10.965,0.234
3240,NaN,Midwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.430,12.800,0.310
3241,NaN,Northeast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.250,14.800,0.530
3242,NaN,South,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.530,11.600,0.280
3243,NaN,Total United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800,13.500,0.340
3244,NaN,West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.230,16.100,0.300


* Now, we will save this combined file, and then generate separate county and state only files...

In [22]:
cnty_df.to_csv('Extra_Output/merged_county_state_region.csv')

In [23]:
# We want to check that there are just 55 excess entries in this table that do not have a county.
len(cnty_df) - cnty_df.county.count()

55

In [24]:
# To generate counties only, make a new DataFrame that drops the na values from county
cnty_only = cnty_df.dropna(subset=['county'])
cnty_only.tail()

,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,obesity,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
3233,Teton,Wyoming,"Teton County, WY",56039.0,21.0,23191.0,90.6,56039.0,14.0,14.0,...,15.0,121.0,4.9,4.0,"Teton County, WY",56039.0,16531.32,1.331,10.965,0.234
3234,Uinta,Wyoming,"Uinta County, WY",56041.0,42.0,20773.0,202.2,56041.0,18.0,32.0,...,13.0,39.0,5.9,9.0,"Uinta County, WY",56041.0,17123.76,1.331,10.965,0.234
3235,Washakie,Wyoming,"Washakie County, WY",56043.0,18.0,8235.0,218.6 (Unreliable),56043.0,16.0,30.0,...,17.0,49.0,4.8,11.0,"Washakie County, WY",56043.0,16171.03,1.331,10.965,0.234
3236,Weston,Wyoming,"Weston County, WY",56045.0,21.0,7236.0,290.2,56045.0,17.0,31.0,...,14.0,41.0,4.1,9.0,"Weston County, WY",56045.0,15923.65,1.331,10.965,0.234
3238,Niobrara,Wyoming,NaN,NaN,NaN,NaN,NaN,56027.0,17.0,25.0,...,16.0,40.0,3.3,8.0,"Niobrara County, WY",56027.0,16054.19,1.331,10.965,0.234


In [25]:
# Save this file
cnty_only.to_csv('merged_counties_only.csv')

In [26]:
# To make only the state file, we will re-make the 'no county' table and then just trim the regions...
nocnty2 = cnty_df.loc[cnty_df.county.isnull(),:]
nocnty2.tail(10)

,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,obesity,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
3045,NaN,Vermont,NaN,NaN,NaN,NaN,NaN,50000.0,17.0,26.0,...,5.0,113.0,6.5,8.0,NaN,NaN,NaN,2.884000,20.660000,0.744000
3084,NaN,Washington,NaN,NaN,NaN,NaN,NaN,53000.0,14.0,28.0,...,7.0,82.0,7.4,9.0,NaN,NaN,NaN,1.783333,18.583333,0.462500
3141,NaN,West Virginia,NaN,NaN,NaN,NaN,NaN,54000.0,25.0,36.0,...,7.0,79.0,9.6,14.0,NaN,NaN,NaN,1.287143,10.990000,0.498571
3213,NaN,Wisconsin,NaN,NaN,NaN,NaN,NaN,55000.0,17.0,31.0,...,6.0,80.0,8.6,9.0,NaN,NaN,NaN,1.477143,12.271429,0.332857
3237,NaN,Wyoming,NaN,NaN,NaN,NaN,NaN,56000.0,19.0,29.0,...,13.0,68.0,4.8,9.0,NaN,NaN,NaN,1.331000,10.965000,0.234000
3240,NaN,Midwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.430000,12.800000,0.310000
3241,NaN,Northeast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.250000,14.800000,0.530000
3242,NaN,South,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.530000,11.600000,0.280000
3243,NaN,Total United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.800000,13.500000,0.340000
3244,NaN,West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.230000,16.100000,0.300000


In [27]:
nocnty2 = nocnty2.drop(index=[3240,3241,3242,3243,3244],axis = 0)
nocnty2 = nocnty2.reset_index()
nocnty2.tail()

,index,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
45,3045,NaN,Vermont,NaN,NaN,NaN,NaN,NaN,50000.0,17.0,...,5.0,113.0,6.5,8.0,NaN,NaN,NaN,2.884000,20.660000,0.744000
46,3084,NaN,Washington,NaN,NaN,NaN,NaN,NaN,53000.0,14.0,...,7.0,82.0,7.4,9.0,NaN,NaN,NaN,1.783333,18.583333,0.462500
47,3141,NaN,West Virginia,NaN,NaN,NaN,NaN,NaN,54000.0,25.0,...,7.0,79.0,9.6,14.0,NaN,NaN,NaN,1.287143,10.990000,0.498571
48,3213,NaN,Wisconsin,NaN,NaN,NaN,NaN,NaN,55000.0,17.0,...,6.0,80.0,8.6,9.0,NaN,NaN,NaN,1.477143,12.271429,0.332857
49,3237,NaN,Wyoming,NaN,NaN,NaN,NaN,NaN,56000.0,19.0,...,13.0,68.0,4.8,9.0,NaN,NaN,NaN,1.331000,10.965000,0.234000


In [28]:
# Need to append DC
nocnty2 = nocnty2.append(cnty_df.iloc[99,:])
nocnty2.tail()

,index,county,state,county_and_state_x,code,Deaths,Population,Crude Rate,fips,smoking,...,noinsurance,physicians,pollution,diabetes,county_and_state_y,code2,sunlight,blow,ganj,smack
46,3084.0,NaN,Washington,NaN,NaN,NaN,NaN,NaN,53000.0,14.0,...,7.0,82.0,7.4,9.0,NaN,NaN,NaN,1.783333,18.583333,0.462500
47,3141.0,NaN,West Virginia,NaN,NaN,NaN,NaN,NaN,54000.0,25.0,...,7.0,79.0,9.6,14.0,NaN,NaN,NaN,1.287143,10.990000,0.498571
48,3213.0,NaN,Wisconsin,NaN,NaN,NaN,NaN,NaN,55000.0,17.0,...,6.0,80.0,8.6,9.0,NaN,NaN,NaN,1.477143,12.271429,0.332857
49,3237.0,NaN,Wyoming,NaN,NaN,NaN,NaN,NaN,56000.0,19.0,...,13.0,68.0,4.8,9.0,NaN,NaN,NaN,1.331000,10.965000,0.234000
99,NaN,"District of Columbia, DC",District of Columbia,"District of Columbia, DC",11001.0,1752.0,681170.0,257.2,NaN,NaN,...,NaN,NaN,NaN,NaN,"District of Columbia, DC",11001.0,15265.36,3.425556,24.177778,0.472222


In [30]:
nocnty2.to_csv('Extra_Output/merged_state_data.csv')

Issues:

* Need to included diabetes data for missing states if possible.
* Need to fill in data at state level
* Need to deal with 'unreliable' death rate estimates (they have a high standard deviation)